# **Data Collecting**

## **NOTE**

This script collects *publicly available* product data from **tiki.vn**,
specifically from category IDs: 
- 8371: Đồng hồ và trang sức
- 1815: Thiết bị số - Phụ kiện số
- 915: Thời trang nam
- 931: Thời trang nữ

Only general product information is collected — including **images, names,
and descriptions** — to serve **educational and demo purposes only**.

⚠️ No personal or user-related data is collected, stored, or used.
The dataset is solely intended for learning, research, and
demonstrating product display functionality on our website.

### **Import libraries**

In [15]:
import requests, csv, time
import os

### **Variable Cofig**

In [ ]:
CATEGORY_ID_LIST = [8371, 1815, 915, 931] 
MAX_PRODUCTS = 2

headers = {
    "User-Agent": "Mozilla/5.0"
}

os.makedirs("data", exist_ok=True)

### **Collecting data**

In [24]:
all_categories = []

for CATEGORY_ID in CATEGORY_ID_LIST:
    items = []
    page = 1

    cat_info = requests.get(f"https://tiki.vn/api/v2/categories/{CATEGORY_ID}", headers=headers).json()
    category_name = cat_info.get("name", f"Category_{CATEGORY_ID}")

    print(f"Collecting products from category: {category_name} (ID: {CATEGORY_ID})")
    while len(items) < MAX_PRODUCTS:
        LIST_API = f"https://tiki.vn/api/v2/products?category={CATEGORY_ID}&page={page}"
        resp = requests.get(LIST_API, headers=headers).json()
        data = resp.get("data", [])
        if not data:
            break

        for p in data:
            pid = p["id"]
            detail = requests.get(f"https://tiki.vn/api/v2/products/{pid}", headers=headers).json()
            desc = detail.get("description", "")

            image_list = [img["base_url"] for img in detail.get("images", []) if "base_url" in img][:5]
            if not image_list:
                image_list = [p.get("thumbnail_url")]

            items.append({
                "id": pid,
                "name": p.get("name"),
                "price": p.get("price"),
                "images": image_list,
                "desc_html": desc
            })

            print(f"GET {len(items)}. {p.get('name')[:60]} ({len(image_list)} ảnh)")
            if len(items) >= MAX_PRODUCTS:
                break
            time.sleep(0.3)

        page += 1
        time.sleep(0.5)

    all_categories.append({
        "category_name": category_name,
        "products": items
    })

with open("data/tiki_products.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["category_name", "id", "name", "price", "images", "desc_html"]
    )
    writer.writeheader()

    for cat in all_categories:
        category_name = cat["category_name"]
        for p in cat["products"]:
            writer.writerow({
                "category_name": category_name,
                "id": p.get("id"),
                "name": p.get("name"),
                "price": p.get("price"),
                "images": ", ".join(p.get("images", [])),
                "desc_html": p.get("desc_html", "").replace("\n", " ")
            })

GET 1. Nhẫn bạc phỉ thủy lục đế vương & lam băng type A hạt 4mm mện (3 ảnh)
GET 2. Pin Tiểu 2A 3A Con Ó Chính Hãng, Bin Đồng Hồ, Pin Xe Đồ Chơi (5 ảnh)
GET 1. Bộ Sạc Siêu Nhanh 25W Type C dành cho Samsung và các dòng đi (3 ảnh)
GET 2. Bộ Sạc Siêu Nhanh 2.0 Type C 45W dành cho Samsung S20/S21/S2 (2 ảnh)
GET 1. Quần lót nam, quần sịp nam tam giác cao cấp ARES 03 chất thu (5 ảnh)
GET 2. Combo 5 quần boxer nam, quần lót nam cao cấp chất thun lạnh  (3 ảnh)
GET 1. Áo Thun Unisex - Oversize Chất 100% Cotton 2 chiều Dày 250gs (1 ảnh)
GET 2. Quần Short Thể Thao Nữ Đa Năng Hàng Việt Nam Nhiều Màu Nhiều (5 ảnh)
